In [ ]:
from utils import make_chat_tree, merge_chat_trees, ChatNode, format_table
import json
from tqdm import tqdm
from typing import List
import os
import pandas as pd
import numpy as np

In [ ]:
all_questions = {}
for file in os.listdir("../data/questions/"):
    question_data = json.load(open("../data/questions/" + file, "r"))
    all_questions[file.split('.')[0]] = question_data

In [ ]:
data_order = json.load(open("../data/all_articles_order.json", "r"))

In [ ]:
all_questions_ordered = []
for file in data_order:
    title = file["article"].split('.')[0]
    all_questions_ordered.append(all_questions[title])
json.dump(all_questions_ordered, open("../data/all_questions_ordered.json", "w"), indent=4)

In [ ]:
for i in range(10):
    all_data = json.load(open("../data/rewrite_" + str(i) + "/all_articles.json", "r"))
    all_answers = []

    j = 0
    for data in tqdm(all_data):
        answers_probe = make_chat_tree("../prompts/ask_questions.json", md_article=data["article"], questions="[" + ",\n".join([q[0] for q in all_questions_ordered[j]]) + "\n]")

        retry = 3
        while retry > 0:
            answers_raw = answers_probe.complete(model="gpt-3.5-turbo-16k-0613", temperature=0.6, max_tokens=1024).content

            answers = []
            try:
                answers = json.loads(("[" + answers_raw.split("[")[1].split("]")[0] +"]"))
                retry = 0
            except:
                retry -= 1
                print(answers_raw)
                print(i, j)
        
        all_answers.append(answers)

        json.dump(all_answers, open("../data/rewrite_" + str(i) + "/all_answers.json", "w"), indent=4)
        j += 1